First, load the data,in this notebook we are using the second Qitta data from the paper "Machine learning–based feature selection to search stable microbial biomarkers: application to inflammatory bowel disease". The data has been preprocessed in R (refer to the r script in iCloud/UTK/GRA-UTK/SelectMicro/MLonMicrobiome).

2) 637 samples: Halfvarson J, Brislawn CJ, Lamendella R, et al. Dynamics of the human gut microbiome in inflammatory bowel disease. Nat Microbiol 2017

The result is from sbatch script (no fine tune):

|                  | number      | Random Forest   | Accuracy       | F1             | MCC            | AUC            |
|----------------|----------------------|--------|----------------|----------------|----------------|----------------|
|AllFeatures    | 400  | |0.757 (0.040)| 0.750 (0.046)| 0.574 (0.079)| 0.887 (0.029)|
| SelectMicro| 57  | |0.744 (0.048)| 0.740 (0.053)| 0.565 (0.100)| 0.889 (0.025) |
| Lasso_finetune    |78  | | 0.757 (0.043)| 0.752 (0.048)| 0.589 (0.081)| 0.882 (0.032)|
| FS_Lasso_finetune |   56 |  |0.730 (0.044)| 0.727 (0.048)| 0.546 (0.071)| 0.890 (0.016) |
| SelectMicro_0.1| 65  | |0.741 (0.032)| 0.737 (0.037)| 0.563 (0.065)| 0.885 (0.023)|
| Lasso_finetune update (lower C)   |59  | | 0.750 (0.049)| 0.747 (0.054)| 0.575 (0.094)| 0.888 (0.032) |
| FS0.1_Lasso_finetune |   44 |  |0.722 (0.052)| 0.720 (0.054)|0.529 (0.097)| 0.884 (0.028)|


|                  | number      | XGboost   | Accuracy       | F1             | MCC            | AUC            |
|----------------|----------------------|--------|----------------|----------------|----------------|----------------|
|AllFeatures    | 400  | |0.721 (0.044)| 0.719 (0.045)| 0.519 (0.071)| 0.871 (0.023) |
| SelectMicro| 57  | |0.719 (0.046)| 0.717 (0.048)| 0.524 (0.072)| 0.878 (0.016) |
| Lasso_finetune    |78  | |  0.708 (0.044)| 0.705 (0.047)| 0.498 (0.082)| 0.871 (0.027) |
| FS_Lasso_finetune |   56 |  | 0.714 (0.045)| 0.712 (0.047)| 0.518 (0.072)| 0.880 (0.017)|
| SelectMicro_0.1| 65  | |0.721 (0.034)| 0.718 (0.038)| 0.527 (0.062)| 0.873 (0.021)|
| Lasso_finetune update (lower C)   |59  | | 0.717 (0.038)|0.715 (0.040)| 0.526 (0.070)| 0.872 (0.023)|
| FS0.1_Lasso_finetune |   44 |  |0.722 (0.045)| 0.719 (0.047)| 0.531 (0.089)| 0.874 (0.020)|




for binary classification of CD and nonIBD, the results is: 

Compared with Lasso, while they have the same accuracy, SelectMicro can predict more true nonIBD which has far less samples compared to CD.


In [ ]:
import sys
sys.path.append('/lustre/isaac24/scratch/mhe8/SelectMicro_24/Code')
import loadData 
import RunML
import FS
import metric

import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

import scikit_posthocs as sp
import itertools
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
count_db1269 = pd.read_csv('../../data/species/data_filtered99/features_db1629.csv',index_col=0)

In [ ]:
meta_db1269 = pd.read_csv('../../data/species/data_filtered99/meta_db1629.csv',index_col=0)
tax_db1269 = pd.read_csv('../../data/species/data_filtered99/tax_db1629.csv',index_col=0)
tax_db1269.index = tax_db1269.index.astype(str)

In [ ]:
print(count_db1269.shape)

In [ ]:
# 1. abundance matrix--> relative abundance matrix
cols_name = count_db1269.columns
data = pd.DataFrame(FS.relative_abundance(count_db1269), columns=cols_name)

In [ ]:
y = meta_db1269['ibd']

In [ ]:
set(y)
# nonIBD: Individuals without IBD (healthy controls).
# CD: Crohn’s Disease.
# UC: Ulcerative Colitis.

In [ ]:
pd.Series(y).value_counts()

Perform sigle lable feature selection, this is done by calculating Kruskal-Wallis H-test on each OTU for each environmental factor, each OTU/ASV is given a weighted h score.

In [ ]:
# 2. calculating H score for each OTU
selectedresult=FS.SelectMicro_fun(data,y,p_cutoff = 0.05)

selectedresult.keys()

In [ ]:
selectedOTU_index= selectedresult['selected_indices']
X_FS = selectedresult['selected_df']


In [ ]:
selected_tax_FS = tax_db1269.loc[X_FS.columns,'Rank']
print(selected_tax_FS)

In [ ]:
# Compute correlation matrix
corr = X_FS.corr(numeric_only=True)

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=False, cmap='coolwarm', fmt=".2f", square=True, linewidths=0.5)
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.show()

In [ ]:
custom_order = ['nonIBD', 'CD', 'UC']
custom_mapping = {category: i for i, category in enumerate(custom_order)}
target_variable = [custom_mapping[category] for category in y]
target_variable = np.array(target_variable)

In [ ]:
X_lasso_ft0,selectedOTU_index_Lasso  = RunML.LassoFS_CV(np.array(data),target_variable)
X_lasso_ft = pd.DataFrame(X_lasso_ft0, columns=cols_name[selectedOTU_index_Lasso])

X_FS_lasso_ft0,xlabel_FS_lasso_ft0  = RunML.LassoFS_CV(np.array(X_FS),target_variable)
selectedOTU_index_FS_lasso = selectedOTU_index[xlabel_FS_lasso_ft0]
X_FS_lasso_ft = pd.DataFrame(X_FS_lasso_ft0, columns=cols_name[selectedOTU_index_FS_lasso])

data_subset = {"AllFeatures":data,
               "SelectMicro": X_FS,
               "Lasso_finetune":X_lasso_ft,
               "FS_Lassofinetune":X_FS_lasso_ft
              }

In [ ]:
print(f'The shape of the original dataset is ',np.shape(data))
print(f'The shape of the SelectMicro dataset is ',np.shape(X_FS))
print(f'The shape of the Lasso_finetune selected dataset is ',np.shape(X_lasso_ft))
print(f'The shape of the FS_Lasso_finetune selected dataset is ',np.shape(X_FS_lasso_ft))

for datatype, subset in data_subset.items():
            print(f"Analysis for {datatype}")
            RunModel.RF_model_SCV_multi(subset, target_variable,SMOTE=True,k=5)


In [ ]:
group = pd.DataFrame({'Disease': y})
group.index = meta_db1269.index
group = group.loc[count_db1269.index]# align disease based on index of count matrix

for datatype, subset in data_subset.items():
    subset.index = count_db1269.index
    metric.plot_heatmap(subset,group)

In [ ]:
for datatype, subset in data_subset.items():
    if datatype != "AllFeatures":
        print(datatype)
        FS.plotPresenseRatio(np.array(subset),y,subset.columns,posLabel="nonIBD",posText="nonIBD",negText="IBD")

In [ ]:
dunn_results, summary = FS.calculate_dunn_tests_pair(
    X_FS,
    target_variable,
    p_threshold=0.05
)

In [ ]:
# Set pandas to display all rows
pd.set_option('display.max_rows', None)
# Assuming your dataframe is named df
print(summary)
# Optionally, reset back to default if needed
pd.reset_option('display.max_rows')

subset = summary[summary['Group1'] == 1]['Feature'].unique()
subset= subset.tolist()
dunn_feature = tax_db1269.loc[tax_db1269.index.isin(subset),'Rank']
print(dunn_feature)


### plot for a feature



In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(dunn_results["4426438"].astype(float), annot=True, cmap='coolwarm', fmt=".4f", cbar_kws={'label': 'p-value'})
plt.title(f"Dunn's Test P-values: 4426438")
plt.tight_layout()
#plt.savefig(os.path.join(output_diry, f"{column}_dunn_heatmap.png"))
plt.show()

In [ ]:
# select some of the features
for feature in subset:
    df_dunntest_x = X_FS[[feature]].copy()
    df_dunntest_x['Group'] = y.values
    plt.figure(figsize=(6, 5))
    sns.boxplot(data=df_dunntest_x, x='Group', y=feature, hue='Group', palette='Set2', legend=False)
    sns.stripplot(data=df_dunntest_x, x='Group', y=feature, color='black', alpha=0.3, jitter=True)
    plt.ylabel("Relative abundance")  # Replace with your desired title
    plt.title(f"{dunn_feature.loc[feature]}")
    plt.tight_layout()
    plt.show()
    print(summary.loc[summary['Feature']==feature,:])

